# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Data Modeling with Apache Cassandra 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS de_project_nosql
    WITH REPLICATION =
    {'class':'SimpleStrategy', 'replication_factor':1}""")
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('de_project_nosql')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

#### Query Model Description:
In order to retrieve data based on session_id and item_in_session filters, we should set session_id as primary key which will be used as partition key, and set item_in_session as clustering key so that the data in the table is organised according to this column. This approach will facilitate efficient querying of the table based on session_id and item_in_session filters.

In [8]:
# Create table with relevant columns
query = "CREATE TABLE IF NOT EXISTS artist_song_by_session"
query = query + "(session_id int, item_in_session int, artist text, song_title text, song_length decimal, PRIMARY KEY(session_id,item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)         

In [9]:
# drop table command
# session.execute("DROP TABLE artist_song_by_session")

In [10]:
# column names in the source file
# ['artist', 'firstName', 'gender', 'itemInSession', 'lastName', 'length', 'level', 'location', 'sessionId', 'song', 'userId']

In [11]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_song_by_session (session_id, item_in_session, artist, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
            

In [50]:
query = "SELECT artist, song_title, song_length FROM artist_song_by_session WHERE session_id = 338 and item_in_session = 4"
t = PrettyTable(['Artist', 'Song', 'Length'])
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    t.add_row([row.artist, row.song_title, row.song_length])
print(t)

+-----------+---------------------------------+----------+
|   Artist  |               Song              |  Length  |
+-----------+---------------------------------+----------+
| Faithless | Music Matters (Mark Knight Dub) | 495.3073 |
+-----------+---------------------------------+----------+


In [34]:
# get result into a dataframe
df1 = pd.DataFrame(list(session.execute(query)))
df1

,artist,song_title,song_length
0,Faithless,Music Matters (Mark Knight Dub),495.3073


In [14]:
# import result into csv file
df1.to_csv('artist_song_by_session.csv', index=False)

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### Query Model Description:
In order to retrieve data based on user_id and session_id sorted by item_in_session, we should set user_id and session_id as primary keys which will form a composite key and be used as partition key in the table. Then we set item_in_session as clustering key so that the data in the table is organised according to this column.

In [15]:
# Create table with relevant columns
query = "CREATE TABLE IF NOT EXISTS artist_song_by_user_id"
query = query + "(user_id int, session_id int, item_in_session int, artist text, song_title text, user_first_name text, user_last_name text, PRIMARY KEY((user_id, session_id), item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)         

In [16]:
# drop table command
# session.execute("DROP TABLE artist_song_by_user_id")

In [17]:
# column names in the source file
# ['artist', 'firstName', 'gender', 'itemInSession', 'lastName', 'length', 'level', 'location', 'sessionId', 'song', 'userId']

In [18]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_song_by_user_id (user_id, session_id, item_in_session, artist, song_title, user_first_name, user_last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [53]:
query = "SELECT artist, song_title, user_first_name, user_last_name FROM artist_song_by_user_id WHERE user_id = 10 and session_id = 182"
t = PrettyTable(['Artist', 'Song', 'User First Name', 'User Last Name'])
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    t.add_row([row.artist, row.song_title, row.user_first_name, row.user_last_name])
print(t)

+-------------------+------------------------------------------------------+-----------------+----------------+
|       Artist      |                         Song                         | User First Name | User Last Name |
+-------------------+------------------------------------------------------+-----------------+----------------+
|  Down To The Bone |                  Keep On Keepin' On                  |      Sylvie     |      Cruz      |
|    Three Drives   |                     Greece 2000                      |      Sylvie     |      Cruz      |
| Sebastien Tellier |                      Kilometer                       |      Sylvie     |      Cruz      |
|   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |      Sylvie     |      Cruz      |
+-------------------+------------------------------------------------------+-----------------+----------------+


In [20]:
# get result into a dataframe
df2 = pd.DataFrame(list(session.execute(query)))
df2

,artist,song_title,user_first_name,user_last_name
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


In [21]:
# import result into csv file
df2.to_csv('artist_song_by_user_id.csv', index=False)

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#### Query Model Description:
In order to retrieve data based on song_title and user names, we should set song_title as primary key and user_id as clustering key. The data will be partitioned by song_title and sorted according to user_id which is the unique identifier for each user.

In [26]:
query = "CREATE TABLE IF NOT EXISTS song_by_user"
query = query + "(song_title text, user_id int, user_first_name text, user_last_name text, PRIMARY KEY(song_title,user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)          

In [27]:
# drop table command
# session.execute("DROP TABLE song_by_user_name")

In [28]:
# column names in source file
# ['artist', 'firstName', 'gender', 'itemInSession', 'lastName', 'length', 'level', 'location', 'sessionId', 'song', 'userId']

In [29]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_by_user (song_title, user_id, user_first_name, user_last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [54]:
query = "SELECT user_first_name, user_last_name FROM song_by_user WHERE song_title = 'All Hands Against His Own'"
t = PrettyTable(['User First Name', 'User Last Name'])
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    t.add_row([row.user_first_name, row.user_last_name])
print(t)

+-----------------+----------------+
| User First Name | User Last Name |
+-----------------+----------------+
|    Jacqueline   |     Lynch      |
|      Tegan      |     Levine     |
|       Sara      |    Johnson     |
+-----------------+----------------+


In [31]:
# get result into a dataframe
df3 = pd.DataFrame(list(session.execute(query)))
df3

,user_first_name,user_last_name
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


In [32]:
# import result into csv file
df3.to_csv('song_by_user.csv', index=False)

### Drop the tables before closing out the sessions

In [55]:
query = "drop table artist_song_by_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [56]:
query = "drop table artist_song_by_user_id"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [57]:
query = "drop table song_by_user"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [58]:
session.shutdown()
cluster.shutdown()